In [ ]:
'''# !pip install geopy
from geopy.geocoders import Nominatim
from tqdm import tqdm  # 진행 상황 확인용 
import time 

def geocoding(address):
    try:
        geolocator = Nominatim(user_agent='South Korea')  
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None  # 좌표를 찾지 못하면 None 반환
    except:
        return None, None
time.sleep(1)'''

In [ ]:
'''start = 2401
end = 3401

# 좌표 결측치인 행만 선택
train_coordinate = train.loc[(train['좌표X'].isna()) | (train['좌표Y'].isna())].copy()
train_coordinate = train_coordinate.reset_index(drop=True)
train_coordinate = train_coordinate.iloc[start:end]'''

In [ ]:
'''for idx, addr in enumerate(tqdm(train_coordinate["도로명"])):
    lat, lon = geocoding(addr)
    train_coordinate.loc[idx, '좌표Y'] = lat
    train_coordinate.loc[idx, '좌표X'] = lon
name = 'imeanseo2401_3401'
# 결과 저장
train_coordinate.to_csv(f"train_xy_{name}.csv", index=False)

train_coordinate.head()'''

In [ ]:
'''train_coordinate.to_csv(f"/root/data_EDA_imeanseo/train_xy_{name}.csv", index=False)'''

In [ ]:
'''
import pandas as pd

df = pd.read_csv("subway_feature.csv")
df.head()
'''

In [ ]:
'''df.head()
df.to_csv("subway_feature_address.csv", index=False)'''


In [ ]:
# 실패
'''import pandas as pd
from geopy.geocoders import Nominatim
import time

# CSV 읽기
df = pd.read_csv("subway_feature.csv")

# 테스트용: 앞에서 50개만 선택
df_test = df.head(50).copy()

# Geopy 역지오코딩 객체
geolocator = Nominatim(user_agent='South Korea')

def reverse_geocoding(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if location:
            return location.address
        else:
            return None
    except:
        return None

# 도로명주소 컬럼에 적용
df_test['도로명주소'] = df_test.progress_apply(
    lambda row: reverse_geocoding(row['위도'], row['경도']),
    axis=1
)

# 결과 확인
print(df_test[['위도', '경도', '도로명주소']])'''

In [ ]:
import pandas as pd

subway = pd.read_csv("subway_feature.csv")
subway.head()

In [ ]:
# import requests
import pandas as pd

url = "https://maps.apigw.ntruss.com/map-reversegeocode/v2/gc"

raddr = []
for x, y in zip(subway['경도'][:2], subway['위도'][:2]):
    params = {
        "coords": f"{x},{y}",  # 경도, 위도
        "output": "json",
        "orders": "legalcode"
    }

    headers = {
        "x-ncp-apigw-api-key-id": id" ",  # Client ID
        "x-ncp-apigw-api-key": api key" "  # Client Secret
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    # 결과 확인용 출력
    print(data)

In [ ]:
# 지하철 data 시군구 뽑기 성공

# import requests
import pandas as pd
import time
from tqdm import tqdm  

# CSV 불러오기
subway = pd.read_csv('subway_feature.csv')

raddr = []  # 법정동 코드 담을 리스트

url = "https://maps.apigw.ntruss.com/map-reversegeocode/v2/gc"

for x, y in tqdm(zip(subway['경도'], subway['위도'])):
    params = {
        "coords": f"{x},{y}",  # 경도, 위도
        "output": "json",
        "orders": "legalcode"
    }

    headers = {
        "x-ncp-apigw-api-key-id": "id",  # Client ID
        "x-ncp-apigw-api-key": "api key"  # Client Secret
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

      # 지역 정보(area1, area2, area3) 추출
    try:
        area1 = data['results'][0]['region']['area1']['name']  # 시/도
        area2 = data['results'][0]['region']['area2']['name']  # 구
        area3 = data['results'][0]['region']['area3']['name']  # 동
        full_address = f"{area1} {area2} {area3}"
    except (KeyError, IndexError):
        full_address = None

    raddr.append(full_address)
    
    # API 과부하 방지
    time.sleep(0.1)

# 데이터프레임에 컬럼 추가
subway['full_address'] = raddr

# CSV로 저장
subway.to_csv('subway_feature_raddr.csv', index=False, encoding='utf-8-sig')

print("subway_feature_raddr.csv 저장 완료")